Взял питоновский датасет от сюда
https://huggingface.co/datasets/bigcode/the-stack/tree/main/data/python
Первые 2000 записей

Нужно закинуть скачать и закинуть файл dataset/train-00000-of-00206.parquet в папку датасет, потом запустить эту штуку.
Далее запустить npm run benchmark
потом можно прогнать анализ


В датасете больше файлов с маленьким размером. 
Для достижения +- равномерного распределения строк по размерам файла, данные сначала делятся на группы по диапазонам размера файла. Затем из каждой группы случайным образом выбирается количество строк, пропорциональное размеру группы, чтобы в сумме получить ровно 1000 строк на пакет. Это обеспечивает более равномерное распределение строк в каждом файле.

In [10]:
import pyarrow.parquet as pq
import pandas as pd
import numpy as np
import json


In [16]:

# Читаем данные из Parquet файла
parquet_file = pq.ParquetFile('dataset/train-00000-of-00206.parquet')
columns_to_read = ['content', 'size']
table = parquet_file.read(columns=columns_to_read)
df = table.to_pandas()

# Ограничение на максимальный размер файла
max_file_size = 20_000
df = df[df['size'] < max_file_size]

# Определяем диапазоны размеров
bins = np.linspace(df['size'].min(), df['size'].max(), num=10)

# Разбиваем DataFrame на группы по размерам
groups = [df[(df['size'] >= bins[i]) & (df['size'] < bins[i+1])] for i in range(len(bins)-1)]

# Параметры пакетов
nrows = 2 * 1000
batch_size = 1000

# Функция для выборки случайных образцов из каждой группы
def sample_batches(groups, batch_size):
    samples = []
    remaining = batch_size
    
    while remaining > 0 and len(groups) > 0:
        group = groups.pop(0)
        if len(group) > 0:
            sample_size = min(len(group), max(1, remaining // len(groups) if len(groups) > 0 else remaining))
            sample = group.sample(n=sample_size)
            samples.append(sample)
            remaining -= sample_size
            groups.append(group.drop(sample.index))  # Удаляем выбранные строки из группы
    
    return pd.concat(samples)

# Генерация и запись файлов пакетов
for i in range(0, nrows, batch_size):
    batch_df = sample_batches(groups.copy(), batch_size)
    
    # Преобразуем DataFrame в список словарей
    data = batch_df.to_dict(orient='records')
    
    # Создаем имя файла для текущего пакета данных
    filename = f'dataset/dataset_{i//batch_size}.json'
    
    # Сериализуем данные в JSON и записываем в файл
    with open(filename, 'w') as json_file:
        json.dump(data, json_file)
        
    print(f'Записан файл {filename} с {len(batch_df)} строками')


Записан файл dataset/dataset_0.json с 1000 строками
Записан файл dataset/dataset_1.json с 1000 строками
